# Dynamic Estimation

This notebook demonstrates rolling and recursive estimation methods, and Autoregressive
Distributed Lag (ADL) models.

**Topics covered:**
- Rolling OLS (direct and from model)
- Recursive OLS (expanding window)
- Rolling AR estimation
- Rolling vs recursive comparison
- Complete workflow: Model → Rolling → Bai-Perron → Model with Breaks
- ADL models (basic, multiple exogenous, lag selection, breaks)
- Rolling and recursive ADL
- Bai-Perron for ADL models

## Setup

In [ ]:
import sys
from pathlib import Path

# Ensure the package is importable when running from the examples/ directory
sys.path.insert(0, str(Path("..") / "src"))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import regimes as rg

print(f"regimes version: {rg.__version__}")

## Generate Regression Data

In [ ]:
np.random.seed(42)
n = 200
break_point = 100

# Generate X (regressor)
X = np.random.randn(n)

# Generate y with different coefficients before/after break
y = np.zeros(n)
y[:break_point] = 1.0 + 0.5 * X[:break_point] + np.random.randn(break_point) * 0.5
y[break_point:] = 3.0 + 1.5 * X[break_point:] + np.random.randn(n - break_point) * 0.5

# Add constant to X for OLS
X_with_const = np.column_stack([np.ones(n), X])

# Plot the data
plt.figure(figsize=(10, 4))
plt.plot(y)
plt.axvline(x=break_point, color='r', linestyle='--', label=f'True break at {break_point}')
plt.xlabel('Observation')
plt.ylabel('y')
plt.title('Simulated Data with Structural Break')
plt.legend()
plt.show()

### Fit a baseline OLS model

In [ ]:
ols_model = rg.OLS(y, X_with_const)
ols_results = ols_model.fit()
print(ols_results.summary())

## Rolling and Recursive Estimation

### 6.1 Rolling OLS â€” Direct Usage

Create a RollingOLS estimator directly with endog, exog, and window size.

In [ ]:
# Direct usage: pass data and window size
rolling_direct = rg.RollingOLS(y, X_with_const, window=60)
rolling_results = rolling_direct.fit()
print(rolling_results.summary())

### 6.2 Rolling OLS â€” From Model

Create rolling estimator from an existing OLS model using `.rolling()` method.

In [ ]:
# From model: create OLS first, then call .rolling()
model = rg.OLS(y, X_with_const, has_constant=False)
rolling_from_model = model.rolling(window=60)
rolling_results_model = rolling_from_model.fit()
print(rolling_results_model.summary())

### 6.3 Recursive OLS â€” Expanding Window

Recursive estimation uses an expanding window that starts with `min_nobs` observations and grows by one at each step.

In [ ]:
# Recursive (expanding window) estimation
model = rg.OLS(y, X_with_const, has_constant=False)
recursive_results = model.recursive(min_nobs=30).fit()
print(recursive_results.summary())

### 6.4 Visualize Rolling Coefficients

Plot coefficient estimates over time with confidence bands. The `plot_coefficients()` method shows how each parameter evolves.

In [ ]:
# Plot rolling coefficients with 95% confidence bands
fig, axes = rolling_results.plot_coefficients(figsize=(10, 6))

# Add vertical line at true break point
for ax in np.ravel(axes):
    ax.axvline(x=break_point, color='r', linestyle='--', alpha=0.7, label='True break')

np.ravel(axes)[0].legend()
plt.tight_layout()
plt.show()

### 6.5 Rolling AR â€” AR Model with Rolling Window

Rolling estimation also works with AR models using the `.rolling()` method.

In [ ]:
# Rolling AR estimation
ar_model = rg.AR(y_ar, lags=1)
ar_rolling = ar_model.rolling(window=60)
ar_rolling_results = ar_rolling.fit()
print(ar_rolling_results.summary())

In [ ]:
# Visualize rolling AR coefficients
fig, axes = ar_rolling_results.plot_coefficients(figsize=(10, 6))

# Add vertical line at true break point (adjusted for effective sample after lags)
for ax in np.ravel(axes):
    ax.axvline(x=break_ar - 1, color='r', linestyle='--', alpha=0.7, label='True break')

np.ravel(axes)[0].legend()
plt.tight_layout()
plt.show()

### 6.6 Compare Rolling vs Recursive Estimation

Both estimation approaches can be compared on the same plot to see how they differ.

In [ ]:
# Compare rolling vs recursive on the same axes
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Plot intercept (x0)
axes[0].plot(rolling_results.params[:, 0], label='Rolling (window=60)', alpha=0.8)
axes[0].plot(recursive_results.params[:, 0], label='Recursive (min_nobs=30)', alpha=0.8)
axes[0].axvline(x=break_point, color='r', linestyle='--', alpha=0.7, label='True break')
axes[0].axhline(y=1.0, color='gray', linestyle=':', alpha=0.5, label='True value (regime 1)')
axes[0].axhline(y=3.0, color='gray', linestyle=':', alpha=0.5, label='True value (regime 2)')
axes[0].set_xlabel('Observation')
axes[0].set_ylabel('Estimate')
axes[0].set_title('Intercept (x0)')
axes[0].legend(fontsize=8)

# Plot slope (x1)
axes[1].plot(rolling_results.params[:, 1], label='Rolling (window=60)', alpha=0.8)
axes[1].plot(recursive_results.params[:, 1], label='Recursive (min_nobs=30)', alpha=0.8)
axes[1].axvline(x=break_point, color='r', linestyle='--', alpha=0.7, label='True break')
axes[1].axhline(y=0.5, color='gray', linestyle=':', alpha=0.5, label='True value (regime 1)')
axes[1].axhline(y=1.5, color='gray', linestyle=':', alpha=0.5, label='True value (regime 2)')
axes[1].set_xlabel('Observation')
axes[1].set_ylabel('Estimate')
axes[1].set_title('Slope (x1)')
axes[1].legend(fontsize=8)

plt.tight_layout()
plt.show()

### 6.7 Complete Workflow: Model â†’ Rolling â†’ Bai-Perron â†’ Model with Breaks

A typical workflow might combine rolling estimation (to visually identify instability) with formal break testing (Bai-Perron) and final estimation with detected breaks.

In [ ]:
# Step 1: Define the base model
model = rg.OLS(y, X_with_const, has_constant=False)

# Step 2: Rolling estimation to visualize potential instability
rolling = model.rolling(window=60).fit()

# Step 3: Formal break test with Bai-Perron
bp_results = model.bai_perron(break_vars="all", max_breaks=3, selection="bic")

# Step 4: Estimate model with detected breaks
final_model = bp_results.to_ols()

print(f"Rolling estimation: {rolling.n_valid} valid windows")
print(f"Bai-Perron detected {bp_results.n_breaks} break(s) at: {bp_results.break_indices}")
print(f"\nFinal model with breaks:")
print(final_model.summary())

In [ ]:
# Visualize the complete workflow
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Top left: Rolling coefficient estimates
ax = axes[0, 0]
ax.plot(rolling.params[:, 0], label='Intercept', alpha=0.8)
ax.plot(rolling.params[:, 1], label='Slope', alpha=0.8)
ax.axvline(x=bp_results.break_indices[0], color='r', linestyle='--', alpha=0.7, label='Detected break')
ax.set_xlabel('Observation')
ax.set_ylabel('Estimate')
ax.set_title('Rolling Parameter Estimates')
ax.legend()

# Top right: Data with detected breaks
ax = axes[0, 1]
rg.plot_breaks(y, results=bp_results, shade_regimes=True, ax=ax)
ax.set_title('Data with Detected Breaks')

# Bottom left: Parameter comparison (intercept)
ax = axes[1, 0]
constant_results = model.fit()
rg.plot_params_over_time(
    {"Constant": constant_results, "With breaks": final_model},
    params=["x0"],
    ax=ax
)
ax.set_title('Intercept: Constant vs Break Model')

# Bottom right: Parameter comparison (slope)
ax = axes[1, 1]
rg.plot_params_over_time(
    {"Constant": constant_results, "With breaks": final_model},
    params=["x1"],
    ax=ax
)
ax.set_title('Slope: Constant vs Break Model')

plt.tight_layout()
plt.show()

## ADL Models

### Generate ADL Data

In [ ]:
np.random.seed(456)
n_adl = 200

# Exogenous regressor
x_adl = np.random.randn(n_adl)

# True parameters
c_true = 1.0      # constant
phi_true = 0.5    # AR(1) coefficient
beta0_true = 0.3  # contemporaneous effect of x
beta1_true = 0.15 # lagged effect of x (L1)

# Simulate ADL(1,1) process
y_adl = np.zeros(n_adl)
for t in range(1, n_adl):
    y_adl[t] = c_true + phi_true * y_adl[t-1] + beta0_true * x_adl[t] + beta1_true * x_adl[t-1] + np.random.randn() * 0.5

# Plot the simulated series
fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

axes[0].plot(y_adl)
axes[0].set_ylabel('y')
axes[0].set_title('Simulated ADL(1,1): Dependent Variable')

axes[1].plot(x_adl, color='orange')
axes[1].set_ylabel('x')
axes[1].set_xlabel('Observation')
axes[1].set_title('Exogenous Regressor')

plt.tight_layout()
plt.show()

### 7.2 Basic ADL Estimation

Fit an ADL(1,1) model â€” one AR lag and one distributed lag on x.

In [ ]:
adl_model = rg.ADL(y_adl, x_adl, lags=1, exog_lags=1)
adl_results = adl_model.fit()
print(adl_results.summary())

print("\nTrue parameters:")
print(f"  const: {c_true}, phi: {phi_true}, beta0: {beta0_true}, beta1: {beta1_true}")

### 7.3 ADL with Multiple Exogenous Variables

When you have multiple exogenous regressors, you can specify:
- **Uniform lag structure**: `exog_lags=2` applies lags 0, 1, 2 to all variables
- **Variable-specific lags**: `exog_lags={"x0": 2, "x1": 0}` allows different lag structures per variable

In [ ]:
# Generate a second exogenous variable
np.random.seed(789)
x1_adl = np.random.randn(n_adl)
X_multi = np.column_stack([x_adl, x1_adl])

# Uniform lag structure: 2 lags for all exog variables
adl_uniform = rg.ADL(y_adl, X_multi, lags=1, exog_lags=2)
results_uniform = adl_uniform.fit()
print("Uniform lag structure (exog_lags=2):")
print(f"Parameter names: {results_uniform.param_names}")
print()

# Variable-specific lag structure
adl_specific = rg.ADL(y_adl, X_multi, lags=1, exog_lags={"x0": 2, "x1": 0})
results_specific = adl_specific.fit()
print("Variable-specific lag structure (x0: 2 lags, x1: contemporaneous only):")
print(f"Parameter names: {results_specific.param_names}")

### 7.4 Distributed Lag Properties

ADL models provide two key measures of the dynamic effect of x on y:

- **Cumulative effect** (Î£Î²): Sum of distributed lag coefficients â€” the total effect of a one-unit change in x on y over all lags
- **Long-run multiplier** (Î£Î² / (1 - Î£Ï†)): The equilibrium effect of a *permanent* unit change in x, accounting for autoregressive dynamics

In [ ]:
# Distributed lag properties from the basic ADL(1,1) model
print("Distributed Lag Properties for ADL(1,1):")
print("-" * 50)
print(f"Cumulative effect:    {adl_results.cumulative_effect}")
print(f"Long-run multiplier:  {adl_results.long_run_multiplier}")

# Compute theoretical values from true parameters
true_cumulative = beta0_true + beta1_true
true_long_run = true_cumulative / (1 - phi_true)
print()
print(f"True cumulative effect:   {true_cumulative}")
print(f"True long-run multiplier: {true_long_run}")

### 7.5 Lag Selection

The `select_lags()` method performs a grid search over AR lags (p) and exog lags (q) to find the optimal specification using AIC or BIC.

In [ ]:
# Create a model and use select_lags to find optimal specification
adl_base = rg.ADL(y_adl, x_adl, lags=1, exog_lags=0)

# Grid search using BIC (default)
optimal_p_bic, optimal_q_bic = adl_base.select_lags(max_ar_lags=4, max_exog_lags=4, criterion="bic")
print(f"Optimal using BIC: ADL({optimal_p_bic},{optimal_q_bic})")

# Grid search using AIC
optimal_p_aic, optimal_q_aic = adl_base.select_lags(max_ar_lags=4, max_exog_lags=4, criterion="aic")
print(f"Optimal using AIC: ADL({optimal_p_aic},{optimal_q_aic})")

print(f"\nTrue specification: ADL(1,1)")

### 7.6 ADL with Structural Breaks

Generate ADL data with a break at observation 100, where the constant and AR coefficient change.

In [ ]:
# Simulate ADL with a break at observation 100
np.random.seed(101)
n_adl_break = 200
break_adl = 100

x_adl_break = np.random.randn(n_adl_break)
y_adl_break = np.zeros(n_adl_break)

# Before break: c=1.0, phi=0.3
# After break: c=2.5, phi=0.6
for t in range(1, n_adl_break):
    if t < break_adl:
        c, phi = 1.0, 0.3
    else:
        c, phi = 2.5, 0.6
    y_adl_break[t] = c + phi * y_adl_break[t-1] + 0.3 * x_adl_break[t] + 0.15 * x_adl_break[t-1] + np.random.randn() * 0.5

# Plot the data
plt.figure(figsize=(10, 4))
plt.plot(y_adl_break)
plt.axvline(x=break_adl, color='r', linestyle='--', label=f'True break at {break_adl}')
plt.xlabel('Observation')
plt.ylabel('y')
plt.title('ADL(1,1) with Structural Break')
plt.legend()
plt.show()

In [ ]:
# ADL with common breaks: all parameters can change at the break
adl_common_break = rg.ADL(y_adl_break, x_adl_break, lags=1, exog_lags=1, breaks=[break_adl])
results_common_break = adl_common_break.fit()
print("ADL with common breaks (all parameters change):")
print(results_common_break.summary())

In [ ]:
# ADL with variable-specific breaks: only intercept changes
adl_variable_break = rg.ADL(y_adl_break, x_adl_break, lags=1, exog_lags=1, 
                            variable_breaks={"const": [break_adl]})
results_variable_break = adl_variable_break.fit()
print("ADL with variable-specific breaks (only constant changes):")
print(results_variable_break.summary())

### 7.7 Rolling and Recursive ADL

Rolling and recursive estimation help visualize parameter stability and can reveal potential structural breaks.

In [ ]:
# Rolling ADL estimation
adl_model_break = rg.ADL(y_adl_break, x_adl_break, lags=1, exog_lags=1)
adl_rolling = adl_model_break.rolling(window=60)
adl_rolling_results = adl_rolling.fit()
print(adl_rolling_results.summary())

In [ ]:
# Plot rolling coefficients
fig, axes = adl_rolling_results.plot_coefficients(figsize=(12, 8))

# Add vertical line at true break point (adjusted for effective sample)
for ax in np.ravel(axes):
    ax.axvline(x=break_adl - 1, color='r', linestyle='--', alpha=0.7, label='True break')

np.ravel(axes)[0].legend()
plt.tight_layout()
plt.show()

In [ ]:
# Recursive ADL estimation
adl_recursive = adl_model_break.recursive(min_nobs=30)
adl_recursive_results = adl_recursive.fit()
print(adl_recursive_results.summary())

In [ ]:
# Compare rolling vs recursive for key parameters
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Intercept comparison
axes[0].plot(adl_rolling_results.params[:, 0], label='Rolling (window=60)', alpha=0.8)
axes[0].plot(adl_recursive_results.params[:, 0], label='Recursive (min_nobs=30)', alpha=0.8)
axes[0].axvline(x=break_adl - 1, color='r', linestyle='--', alpha=0.7, label='True break')
axes[0].axhline(y=1.0, color='gray', linestyle=':', alpha=0.5)
axes[0].axhline(y=2.5, color='gray', linestyle=':', alpha=0.5)
axes[0].set_xlabel('Observation')
axes[0].set_ylabel('Estimate')
axes[0].set_title('Intercept')
axes[0].legend(fontsize=8)

# AR(1) coefficient comparison
axes[1].plot(adl_rolling_results.params[:, 1], label='Rolling (window=60)', alpha=0.8)
axes[1].plot(adl_recursive_results.params[:, 1], label='Recursive (min_nobs=30)', alpha=0.8)
axes[1].axvline(x=break_adl - 1, color='r', linestyle='--', alpha=0.7, label='True break')
axes[1].axhline(y=0.3, color='gray', linestyle=':', alpha=0.5)
axes[1].axhline(y=0.6, color='gray', linestyle=':', alpha=0.5)
axes[1].set_xlabel('Observation')
axes[1].set_ylabel('Estimate')
axes[1].set_title('AR(1) Coefficient')
axes[1].legend(fontsize=8)

plt.tight_layout()
plt.show()

### 7.8 Bai-Perron Test for ADL

The `model.bai_perron()` method runs a formal structural break test on the ADL model.

In [ ]:
# Run Bai-Perron test on ADL model
adl_test_model = rg.ADL(y_adl_break, x_adl_break, lags=1, exog_lags=1)
bp_adl_results = adl_test_model.bai_perron(break_vars="all", max_breaks=3, selection="bic")
print(f"Detected breaks: {bp_adl_results.break_indices}")
print(f"True break: {break_adl}")
print()
print(bp_adl_results.summary())

In [ ]:
# Complete workflow: ADL â†’ Bai-Perron â†’ ADL with detected breaks
print("Complete ADL Workflow: Constant Model â†’ Break Test â†’ Model with Breaks")
print("=" * 70)

# Step 1: Fit ADL without breaks
adl_constant = rg.ADL(y_adl_break, x_adl_break, lags=1, exog_lags=1)
constant_result = adl_constant.fit()
print(f"\nStep 1: Constant ADL(1,1)")
print(f"  AIC: {constant_result.aic:.2f}, BIC: {constant_result.bic:.2f}")

# Step 2: Test for breaks
bp_result = adl_constant.bai_perron(break_vars="all", max_breaks=3)
print(f"\nStep 2: Bai-Perron Test")
print(f"  Detected {bp_result.n_breaks} break(s) at: {bp_result.break_indices}")

# Step 3: Fit ADL with detected breaks
if bp_result.break_indices:
    adl_with_breaks = rg.ADL(y_adl_break, x_adl_break, lags=1, exog_lags=1, 
                              breaks=bp_result.break_indices)
    break_result = adl_with_breaks.fit()
    print(f"\nStep 3: ADL with Detected Breaks")
    print(f"  AIC: {break_result.aic:.2f}, BIC: {break_result.bic:.2f}")
    print()
    print(break_result.summary())

In [ ]:
# Visualize the complete workflow
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Top left: Rolling coefficients for intercept and AR(1)
ax = axes[0, 0]
ax.plot(adl_rolling_results.params[:, 0], label='Intercept', alpha=0.8)
ax.plot(adl_rolling_results.params[:, 1], label='y.L1', alpha=0.8)
ax.axvline(x=break_adl - 1, color='r', linestyle='--', alpha=0.7, label='True break')
ax.set_xlabel('Observation')
ax.set_ylabel('Estimate')
ax.set_title('Rolling ADL Coefficients')
ax.legend()

# Top right: Data with detected breaks
ax = axes[0, 1]
ax.plot(y_adl_break, alpha=0.8)
ax.axvline(x=break_adl, color='r', linestyle='--', alpha=0.7, label='True break')
for bp in bp_result.break_indices:
    ax.axvline(x=bp, color='blue', linestyle=':', alpha=0.7, label='Detected break')
ax.set_xlabel('Observation')
ax.set_ylabel('y')
ax.set_title('Data with True and Detected Breaks')
ax.legend()

# Bottom: Distributed lag effects by regime
ax = axes[1, 0]
if bp_result.break_indices:
    regimes = ["Regime 1", "Regime 2"]
    cum_effects = []
    lr_multipliers = []
    
    # Extract coefficients from the model with breaks
    params = break_result.params
    param_names = break_result.param_names
    
    # Find x0 and x0.L1 for each regime
    for r in [1, 2]:
        x0_idx = param_names.index(f"x0_regime{r}")
        x0l1_idx = param_names.index(f"x0.L1_regime{r}")
        yl1_idx = param_names.index(f"y.L1_regime{r}")
        
        cum = params[x0_idx] + params[x0l1_idx]
        lr = cum / (1 - params[yl1_idx])
        cum_effects.append(cum)
        lr_multipliers.append(lr)
    
    x = np.arange(len(regimes))
    width = 0.35
    ax.bar(x - width/2, cum_effects, width, label='Cumulative Effect')
    ax.bar(x + width/2, lr_multipliers, width, label='Long-run Multiplier')
    ax.set_xticks(x)
    ax.set_xticklabels(regimes)
    ax.set_ylabel('Effect')
    ax.set_title('Distributed Lag Effects by Regime')
    ax.legend()

# Bottom right: Model comparison via information criteria
ax = axes[1, 1]
models = ['Constant ADL', 'ADL with Breaks']
aics = [constant_result.aic, break_result.aic]
bics = [constant_result.bic, break_result.bic]
x = np.arange(len(models))
width = 0.35
ax.bar(x - width/2, aics, width, label='AIC')
ax.bar(x + width/2, bics, width, label='BIC')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.set_ylabel('Information Criterion')
ax.set_title('Model Comparison')
ax.legend()

plt.tight_layout()
plt.show()